# 1 Введение

## 1.1 Постановка задачи

**Заказчик** -- банк
  
**Задача** -- по предоставленным историческим данным о поведении клиентов и расторжении ими договоров с банком построить модель машинного обучения (ML), которая должна прогнозировать уйдёт клиент из банка в ближайшее время или нет. Метрикой качества работы модели ML принять F1-меру, значение которой должно быть не ниже 0.59

## 1.2 Исходные данные

### 1.2.1 Переданные файлы
- Churn.csv

### 1.2.2 Описание данных
**Признаки**
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — количество недвижимости у клиента
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата  

**Целевой признак**
- Exited — факт ухода клиента

## 1.3 План обработки и анализа данных

### 1.3.1 Предобработка данных
- обзор данных;
- корректировка имён столбцов (при необходимости);
- обнаружение и удаление пропусков:
 - выявить пропуски;
 - проанализировать причины их появления;
 - заполнить пропуски (приняв решение по алгоритму заполнения), либо удалить их;
- преобразование типов данных столбцов (при необходимости);
- поиск и исправление ошибок (аномальных значений) в данных;
- удаление дубликатов строк.

### 1.3.2 Подготовка признаков и наборов
- прямое кодирование (при необходимости);
- порядковое кодирование (при необходимости);
- разбиение данных на обучающую, валидационную и тестовые выборки, выделение целового признака;
- масштабирование признаков (при необходимости).

### 1.3.3 Исследование задачи
- исследование баланса классов;
- создание моделей ML и оценка их качества.

### 1.3.4 Удаление дисбаланса
Удаление дисбаланса (при необходимости):
- взвешивание классов;
- увеличение выборок (*upsampling*);
- уменьшение выборок (*downsamoling*).

### 1.3.5 Тестирование модели
На тестовой выборке выполнить:
- расчёт F1-меры;
- расчёт AUC-ROC.

### 1.3.6 Вывод

## 1.4 Пользовательские функции

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

from sklearn.utils import shuffle

pd.options.display.float_format = '{:.2f}'.format

In [2]:
def find_model(features_train, target_train, features_valid, target_valid,
               model_type='', goal='', max_depth=1, n_estimators=100, class_weight=None):
    '''
    Функция поиска оптимальных гиперпараметров для моделей логистической регрессии, решающего дерева и случайного леса

    model_type = log_reg
                 tree
                 forest
    goal = calc
           RP
    '''
    
    if goal == 'calc':
        
        if model_type == 'log_reg':
            
            model = LogisticRegression(random_state=1456, solver='liblinear', class_weight=class_weight)
            model.fit(features_train, target_train)
            predicted = model.predict(features_valid)
            
            ROC_AUC = roc_auc_score(target_valid, predicted)
            f1 = f1_score(target_valid,predicted)

            print("ROC-AUC:", ROC_AUC)
            print("F1 мера:", f1)
            
            return ROC_AUC, f1
            
        elif model_type == 'tree':
            
            max_ROC_AUC = [0, 0]
            max_f1 = [0, 0]

            for i in range(1, 100):
                model = DecisionTreeClassifier(random_state=1456, max_depth=i, class_weight=class_weight)

                model.fit(features_train, target_train)
                predicted = model.predict(features_valid)
                
                if roc_auc_score(target_valid, predicted) > max_ROC_AUC[0]:
                    max_ROC_AUC[0] = roc_auc_score(target_valid, predicted)
                    max_ROC_AUC[1] = i
                    
                if f1_score(target_valid, predicted) > max_f1[0]:
                    max_f1[0] = f1_score(target_valid, predicted)
                    max_f1[1] = i
        
            print('[Max ROC-AUC, max_depth]:', max_ROC_AUC)
            print('[Max F1, max_depth]:', max_f1)
            
            return max_ROC_AUC[0], max_f1[0]          

        elif model_type == 'forest':
            
            max_ROC_AUC = [0, 0, 0]
            max_f1 = [0, 0, 0]

            for i in range(5, 30):
                for j in range(1, 100, 3):
                    model = RandomForestClassifier(random_state=1456, max_depth=i, n_estimators=j,
                                               class_weight=class_weight)

                model.fit(features_train, target_train)
                predicted = model.predict(features_valid)
                
                if roc_auc_score(target_valid, predicted) > max_ROC_AUC[0]:
                    max_ROC_AUC[0] = roc_auc_score(target_valid, predicted)
                    max_ROC_AUC[1] = i
                    max_ROC_AUC[2] = j
                    
                if f1_score(target_valid, predicted) > max_f1[0]:
                    max_f1[0] = f1_score(target_valid, predicted)
                    max_f1[1] = i
                    max_f1[2] = j
        
            print('[Max ROC-AUC, max_depth, n_estimators]:', max_ROC_AUC)
            print('[Max F1, max_depth, n_estimators]:', max_f1)
            
            return max_ROC_AUC[0], max_f1[0]
        
        else: 
            print('Wrong model type')
            
    elif  goal == 'RP':
        
        if model_type == 'log_reg':
            model = LogisticRegression(random_state=1456, solver='liblinear', class_weight=class_weight)
        elif model_type == 'tree':
            model = DecisionTreeClassifier(random_state=1456, max_depth=max_depth, class_weight=class_weight)
        elif model_type == 'forest':
            model = RandomForestClassifier(random_state=1456, max_depth=max_depth, n_estimators=n_estimators, 
                                           class_weight=class_weight)
        else: 
            print('Wrong model type')
            return
        
        model.fit(features_train, target_train)
        predicted = model.predict(features_valid)

        R = recall_score(target_valid, predicted)
        P = precision_score(target_valid,predicted)
        
        print("Полнота (recall):", R)
        print("Точность (precision):", P)
        
        return R, P
            
    else:
        print('Wrong goal')

In [3]:
def upsample(features, target, repeat):
    '''
    Функция проводящая Upsample набора данных с бинарным целевым признаком
    '''
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=1456)
    
    return features_upsampled, target_upsampled

In [4]:
def downsample(features, target, fraction):
    '''
    Функция проводящая Downsample набора данных с бинарным целевым признаком
    '''
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=1456)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=1456)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=1456)
    
    return features_downsampled, target_downsampled

In [5]:
def test_model(features_train, target_train, features_test, target_test,
               model_type='', max_depth=1, n_estimators=100, class_weight=None):
    '''
    Функция проверки модели на тестовых данных
    
    model_type = log_reg
                 tree
                 forest
    '''
    
    if model_type == 'log_reg':
        model = LogisticRegression(random_state=1456, solver='liblinear', class_weight=class_weight)
        
    elif model_type == 'tree':
        model = DecisionTreeClassifier(random_state=1456, max_depth=max_depth, class_weight=class_weight)
        
    elif model_type == 'forest':
        model = RandomForestClassifier(random_state=1456, max_depth=max_depth, n_estimators=n_estimators, 
                                       class_weight=class_weight)
        
    else: 
            print('Wrong model type')
            return
    
    model.fit(features_train, target_train)
    predicted = model.predict(features_test)
            
    print('F1-мера: {:.3f}'.format(f1_score(target_test,predicted)))
    print('ROC-AUC: {:.3f}'.format(roc_auc_score(target_test, predicted)))
    print('recall (полнота): {:.3f}'.format(recall_score(target_test, predicted)))
    print('precision (точность): {:.3f}'.format(precision_score(target_test,predicted)))

# 2 Основная часть

## 2.1 Предобработка данных

**Обзор данных**

In [6]:
raw_data = pd.read_csv('Churn.csv')

In [7]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [8]:
raw_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.00,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.00,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.00,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.00,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.00,125510.82,1,1,1,79084.10,0


In [9]:
raw_data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00,10000.00,10000.00,10000.00,9091.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00
mean,5000.50,15690940.57,650.53,38.92,5.00,76485.89,1.53,0.71,0.52,100090.24,0.20
std,2886.90,71936.19,96.65,10.49,2.89,62397.41,0.58,0.46,0.50,57510.49,0.40
min,1.00,15565701.00,350.00,18.00,0.00,0.00,1.00,0.00,0.00,11.58,0.00
25%,2500.75,15628528.25,584.00,32.00,2.00,0.00,1.00,0.00,0.00,51002.11,0.00
50%,5000.50,15690738.00,652.00,37.00,5.00,97198.54,1.00,1.00,1.00,100193.91,0.00
75%,7500.25,15753233.75,718.00,44.00,7.00,127644.24,2.00,1.00,1.00,149388.25,0.00
max,10000.00,15815690.00,850.00,92.00,10.00,250898.09,4.00,1.00,1.00,199992.48,1.00


In [10]:
raw_data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [11]:
raw_data['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [12]:
raw_data.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

По итогам первого, общего взгляда на данные, можно сделать следующие выводы:

- набор данных состоит из 14 столбцов и 10000 строк;
- имена столбцов читабельны, но требуют приведения их к привычному виду ("under_score" нотации); имя столбца **Tenure** не соответствувет данным, хранящимся в нём; столбец неоходимо переименовать в **realty_count**;
- столбец **row_number** дублирует по своей сути **raw_data.index**, поэтому может быть удалён из набора данных;  
- столбцы **surname** и **customer_id** персонифицируют данные и для обучения модели ML ценности не представляют, поэтому их необходимо удалить;
- пропуски имеются в столбце **Tenure**, которые будем воспринимать, как отсутствие недвижимости у клиента, т.е. равное 0;
- аномалии в данных отсутствуют.  

**Удаление столбцов**

In [13]:
raw_data = raw_data.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis='columns')

**Корректировка имён столбцов**

In [14]:
# Приведение к нижнему регистру
raw_data.columns = raw_data.columns.str.lower()

# Переименование столбцов
raw_data.rename(columns={
                'creditscore': 'credit_score',
                'tenure': 'realty_count',
                'numofproducts': 'num_of_products',
                'hascrcard': 'has_cr_card',
                'isactivemember': 'is_active_member',
                'estimatedsalary': 'estimated_salary'}, 
                inplace=True)

raw_data.columns

Index(['credit_score', 'geography', 'gender', 'age', 'realty_count', 'balance',
       'num_of_products', 'has_cr_card', 'is_active_member',
       'estimated_salary', 'exited'],
      dtype='object')

**Удаление пропусков**

In [15]:
raw_data.fillna(0, inplace=True)
raw_data.isna().sum()

credit_score        0
geography           0
gender              0
age                 0
realty_count        0
balance             0
num_of_products     0
has_cr_card         0
is_active_member    0
estimated_salary    0
exited              0
dtype: int64

In [16]:
raw_data.head()

,credit_score,geography,gender,age,realty_count,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.00,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.00,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.00,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.00,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.00,125510.82,1,1,1,79084.10,0


**Удаление дубликатов строк**

In [17]:
raw_data.duplicated().sum()

0

Дубликаты строк отсутствуют

## 2.2 Подготовка признаков и наборов

**Прямое кодирование (*One-Hot Encoding*)**  
  
ОНЕ-кодирование применим для столбцов **geography**, **gender**. Исключим возможность возникноваения *dummy*-ловушек

In [18]:
ohe_data = pd.get_dummies(raw_data, columns=['geography', 'gender'], drop_first=True)
ohe_data.head()

,credit_score,age,realty_count,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.00,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.00,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.00,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.00,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.00,125510.82,1,1,1,79084.10,0,0,1,0


**Разделение набора данных, выделение целевого признака**

Выделим из переданного набора данных обучающую, валидационную и тестовую выборки в соотношении 3:1:1 (0.6:0.2:0.2) соответственно. Для этого начальную выборку разделим на две выборки в соотнощении 0.6:0.4, а затем выборку 0.4 разделим на выборки в соотношении 0.5:0.5

In [19]:
train, valid = train_test_split(ohe_data, test_size=0.4, random_state=1456)
valid, test = train_test_split(valid, test_size=0.5, random_state=1456)

In [20]:
# Создание обучающей выборки
target_train = train['exited']
features_train = train.drop(['exited'], axis=1)

# Создание валидационной выборки
target_valid = valid['exited']
features_valid = valid.drop(['exited'], axis=1)

# Создание тестовой выборки
target_test = test['exited']
features_test= test.drop(['exited'], axis=1) # переменные - признаки (тестовые данные)

In [21]:
features_train.shape

(6000, 11)

In [22]:
features_valid.shape

(2000, 11)

In [23]:
features_test.shape

(2000, 11)

**Масштабирование признаков**

In [24]:
numeric = ['credit_score','age','realty_count','balance','num_of_products','estimated_salary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
 
features_train[numeric] = scaler.transform(features_train[numeric])
features_train.head()

,credit_score,age,realty_count,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
7050,-0.62,-0.95,0.45,-1.24,0.81,1,1,0.16,0,0,1
676,-0.60,-0.08,-0.19,0.19,0.81,1,0,-0.56,0,1,0
9014,-0.57,2.02,0.77,0.72,-0.91,0,1,-1.20,0,1,1
8014,0.83,0.01,-0.83,0.80,-0.91,1,1,-0.42,0,1,0
8605,-0.93,0.97,1.10,-1.24,0.81,1,1,-1.51,0,0,0


In [25]:
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_valid.head()

,credit_score,age,realty_count,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
4034,-0.51,0.30,-0.19,0.32,-0.91,1,1,1.74,0,1,0
8555,-1.33,-1.33,-1.15,0.55,-0.91,1,1,1.21,1,0,1
9767,-1.00,-0.56,-0.51,0.65,-0.91,1,1,-1.50,0,0,1
3427,-1.92,0.20,1.42,0.64,-0.91,1,0,1.20,0,0,1
3520,-1.00,0.40,1.74,-1.24,0.81,1,0,0.87,0,0,0


In [26]:
features_test[numeric] = scaler.transform(features_test[numeric])
features_test.head()

,credit_score,age,realty_count,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
7775,-0.23,0.11,1.74,-1.24,0.81,1,0,0.08,0,1,1
2913,1.41,0.11,0.45,-1.24,0.81,1,1,-0.26,0,0,0
4687,1.22,0.87,-0.83,0.31,0.81,0,1,1.71,0,0,1
5116,0.94,-0.08,-0.19,0.82,-0.91,1,0,-0.71,1,0,1
769,-1.24,2.31,-1.15,0.60,-0.91,0,1,-1.30,0,0,0


***Промежуточный итог*** 
  
По итогам этапа предобработки данных:
- имена столбцов набора данных приведены к "under_score" нотации;
- переименован столбец **Tenure** в **realty_count**, в нём удалены пропуски;
- удалены столбцы **row_number**, **surname**, **customer_id**;
- проведено OHE-кодирование для столбцов **geography**, **gender**;
- исходная выборка разделена на обучающую, валидационную и тестовую выборки в соотношении 3:1:1 (0.6:0.2:0.2) соответственно, выделены из них целевые прзнаки;
- проведено масштабирование значений стобцов **credit_score**, **age**, **realty_count**, **balance**, **num_of_products**, **estimated_salary** всех выборок.
  
Аномальные значения и дубликаты строк в переданном наборе данных отсутствуют.

## 2.3 Исследование задачи

**Исследование баланса классов**

In [27]:
target_train.value_counts(normalize=True)

0   0.80
1   0.20
Name: exited, dtype: float64

Размер положительного класса целевого признака значительно меньше рамера отрицательного. Клиенты Бета-Банку доверяют и редко от него уходят. Но именно они (ушедшие) нас и интересуют.  
  
Построим несколько моделей ML на несбалнсированных данных и оценим их качество.  
Целевой признак бинарен, поэтому будем строить модели <u>логистической регрессии</u>, <u>решающего дерева</u> и <u>случайного леса</u>.

**Логистическая регрессия**

In [28]:
log_reg_results = pd.DataFrame(columns=['unbalanced', 'auto_balance', 'upsamle', 'downsample'],
                               index=['AUC-ROC', 'F1', 'recall', 'precision'])

In [29]:
results = find_model(features_train, target_train, features_valid, target_valid, model_type='log_reg', goal='calc')

log_reg_results.loc['AUC-ROC', 'unbalanced'] = results[0]
log_reg_results.loc['F1', 'unbalanced'] = results[1]

ROC-AUC: 0.5945958056999462
F1 мера: 0.3264540337711069


Значение ROC-AUC чуть выше 0.5 говорит о том, что ROC-линия модели расопложена чуть выше ROC-линии слуайного классификатора, т.е. о низком качестве модели. F1-мера далека от 1, что говорит о низком значении полноты или точности. Проверм их.

In [30]:
results = find_model(features_train, target_train, features_valid, target_valid, model_type='log_reg', goal='RP')

log_reg_results.loc['recall', 'unbalanced'] = results[0]
log_reg_results.loc['precision', 'unbalanced'] = results[1]

Полнота (recall): 0.21428571428571427
Точность (precision): 0.6850393700787402


У модели плохая полнота -- много FN значений, т.е. модель оценивает уход клинета как 0, хотя на самом деле его уход равен 1. Плохо, совсем не то, что ждёт Заказчик от модели. Возможно это связано с дисбалансом классов целевого признака и это, как следствие, плохой результат обучения модели

**Решающее дерево**

In [31]:
tree_results = pd.DataFrame(columns=['unbalanced', 'auto_balance', 'upsamle', 'downsample'],
                               index=['AUC-ROC', 'F1', 'recall', 'precision'])

In [32]:
results = find_model(features_train, target_train, features_valid, target_valid, model_type='tree', goal='calc')

tree_results.loc['AUC-ROC', 'unbalanced'] = results[0]
tree_results.loc['F1', 'unbalanced'] = results[1]

[Max ROC-AUC, max_depth]: [0.7287395466991367, 7]
[Max F1, max_depth]: [0.6042296072507553, 7]


У полученной модели хорошее значение ROC-AUC и не плохое значение F1-меры. Проверим полноту с точностью

In [33]:
results = find_model(features_train, target_train, features_valid, target_valid, model_type='tree', goal='RP', max_depth=7)

tree_results.loc['recall', 'unbalanced'] = results[0]
tree_results.loc['precision', 'unbalanced'] = results[1]

Полнота (recall): 0.49261083743842365
Точность (precision): 0.78125


И снова у модели плохая полнота. Продолжаем исследование

**Случайный лес**

In [34]:
forest_results = pd.DataFrame(columns=['unbalanced', 'auto_balance', 'upsamle', 'downsample'],
                               index=['AUC-ROC', 'F1', 'recall', 'precision'])

In [35]:
results = find_model(features_train, target_train, features_valid, target_valid, model_type='forest', goal='calc')

forest_results.loc['AUC-ROC', 'unbalanced'] = results[0]
forest_results.loc['F1', 'unbalanced'] = results[1]

[Max ROC-AUC, max_depth, n_estimators]: [0.72970375360805, 16, 97]
[Max F1, max_depth, n_estimators]: [0.609375, 13, 97]


У полученной модели хорошее значение ROC-AUC и не плохое значение F1-меры. Проверим полноту с точностью

In [36]:
results = find_model(features_train, target_train, features_valid, target_valid,
                     model_type='forest', goal='RP', max_depth=14, n_estimators=97)

forest_results.loc['recall', 'unbalanced'] = results[0]
forest_results.loc['precision', 'unbalanced'] = results[1]

Полнота (recall): 0.4852216748768473
Точность (precision): 0.8040816326530612


Полнота модели случайного леса хуже, чем у решающего дерева

***Промежуточный итог*** 
  
Полученные модели ML, построенные на несбалансированных классах целевого признака, обладают низкой полнотой, т.е. в результатах их работы много FN значений -- самый негативный вариант работы модели для Заказчика, когда модель прогнозирует, что клиент не уйдёт, а по факут он уходит.  
  
Необходимо улучшать качество работы модели. Для этого уберём дисбаланс классов целевого признака

## 2.4 Удаление дисбаланса

Дисбаланс постараемся убрать при помощи:
- автоматической балансировки классов  
  или
- upsamle положительного класса;  
  или
- downsample отрицательного класса.  
  
Преобразовывать будем обучающйю выборку с проверкой метрик AUC-ROC, F1 для моделей логистической регрессии, решающего дерева  и случайного леса

### 2.4.1 Автоматическая балансировка классов

**Логистическая регрессия**

In [37]:
results = find_model(features_train, target_train, features_valid, target_valid,
                     model_type='log_reg', goal='calc', class_weight='balanced')

log_reg_results.loc['AUC-ROC', 'auto_balance'] = results[0]
log_reg_results.loc['F1', 'auto_balance'] = results[1]

ROC-AUC: 0.7258407451588779
F1 мера: 0.5164644714038129


In [38]:
log_reg_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.59,0.73,NaN,NaN
F1,0.33,0.52,NaN,NaN
recall,0.21,NaN,NaN,NaN
precision,0.69,NaN,NaN,NaN


Качество модели заметно выросло. Проверим полноту и точность.

In [39]:
results = find_model(features_train, target_train, features_valid, target_valid,
                     model_type='log_reg', goal='RP', class_weight='balanced')

log_reg_results.loc['recall', 'auto_balance'] = results[0]
log_reg_results.loc['precision', 'auto_balance'] = results[1]

Полнота (recall): 0.7339901477832512
Точность (precision): 0.3983957219251337


In [40]:
log_reg_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.59,0.73,NaN,NaN
F1,0.33,0.52,NaN,NaN
recall,0.21,0.73,NaN,NaN
precision,0.69,0.40,NaN,NaN


Полнота модели приятно выросла, т.е. значений FN стало заметно меньше. Однако точность сильно упала -- модель стала "пугливой", стало много значений FP. 

**Решающее дерево**

In [41]:
results = find_model(features_train, target_train, features_valid, target_valid,
                     model_type='tree', goal='calc', class_weight='balanced')

tree_results.loc['AUC-ROC', 'auto_balance'] = results[0]
tree_results.loc['F1', 'auto_balance'] = results[1]

[Max ROC-AUC, max_depth]: [0.7824662682102219, 6]
[Max F1, max_depth]: [0.5932835820895522, 6]


In [42]:
tree_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.78,NaN,NaN
F1,0.60,0.59,NaN,NaN
recall,0.49,NaN,NaN,NaN
precision,0.78,NaN,NaN,NaN


Качество модели практически не изменилось. Проверим полноту и точность.

In [43]:
results = find_model(features_train, target_train, features_valid, target_valid,
                     model_type='tree', goal='RP', class_weight='balanced', max_depth=6)

tree_results.loc['recall', 'auto_balance'] = results[0]
tree_results.loc['precision', 'auto_balance'] = results[1]

Полнота (recall): 0.7832512315270936
Точность (precision): 0.4774774774774775


In [44]:
tree_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.78,NaN,NaN
F1,0.60,0.59,NaN,NaN
recall,0.49,0.78,NaN,NaN
precision,0.78,0.48,NaN,NaN


Аналогично логистической регрессии полнота решающего дерева выросла с одовременным снижением точности. Модель также стала "пугливой"

**Случайный лес**

In [45]:
results = find_model(features_train, target_train, features_valid, target_valid, 
                     model_type='forest', goal='calc', class_weight='balanced')

forest_results.loc['AUC-ROC', 'auto_balance'] = results[0]
forest_results.loc['F1', 'auto_balance'] = results[1]

[Max ROC-AUC, max_depth, n_estimators]: [0.8015881600336237, 7, 97]
[Max F1, max_depth, n_estimators]: [0.654503990877993, 9, 97]


In [46]:
forest_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.80,NaN,NaN
F1,0.61,0.65,NaN,NaN
recall,0.49,NaN,NaN,NaN
precision,0.80,NaN,NaN,NaN


Качество модели на сбалансированных классах значительно улучшилось. Проверим полноту и точность

In [47]:
results = find_model(features_train, target_train, features_valid, target_valid,
                     model_type='forest', goal='RP', class_weight='balanced', max_depth=9, n_estimators=97)

forest_results.loc['recall', 'auto_balance'] = results[0]
forest_results.loc['precision', 'auto_balance'] = results[1]

Полнота (recall): 0.7068965517241379
Точность (precision): 0.6093418259023354


In [48]:
forest_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.80,NaN,NaN
F1,0.61,0.65,NaN,NaN
recall,0.49,0.71,NaN,NaN
precision,0.80,0.61,NaN,NaN


Хорошая полнота, хорошая точность (как следствие -- хорошее значение f1-меры) и хорошая AUC-ROC позволяют признать модель хорошей

### 2.4.2 Upsample положительного класса

In [49]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

**Логистическая регрессия**

In [50]:
results = find_model(features_upsampled, target_upsampled, features_valid, target_valid, model_type='log_reg', goal='calc')

log_reg_results.loc['AUC-ROC', 'upsamle'] = results[0]
log_reg_results.loc['F1', 'upsamle'] = results[1]

ROC-AUC: 0.6605080010630999
F1 мера: 0.43009545199326216


In [51]:
log_reg_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.59,0.73,0.66,NaN
F1,0.33,0.52,0.43,NaN
recall,0.21,0.73,NaN,NaN
precision,0.69,0.40,NaN,NaN


In [52]:
results = find_model(features_upsampled, target_upsampled, features_valid, target_valid, model_type='log_reg', goal='RP')

log_reg_results.loc['recall', 'upsamle'] = results[0]
log_reg_results.loc['precision', 'upsamle'] = results[1]

Полнота (recall): 0.9433497536945813
Точность (precision): 0.27854545454545454


In [53]:
log_reg_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.59,0.73,0.66,NaN
F1,0.33,0.52,0.43,NaN
recall,0.21,0.73,0.94,NaN
precision,0.69,0.40,0.28,NaN


Upsample для логистической регрессии качество модели не улучшил -- F1-мера уменьшилась в сравнении с автоматической балансировкой, что вызвано очень сильным ухудшением точности

**Решающее дерево**

In [54]:
results = find_model(features_upsampled, target_upsampled, features_valid, target_valid, model_type='tree', goal='calc')

tree_results.loc['AUC-ROC', 'upsamle'] = results[0]
tree_results.loc['F1', 'upsamle'] = results[1]

[Max ROC-AUC, max_depth]: [0.746710262004685, 5]
[Max F1, max_depth]: [0.5374449339207048, 11]


In [55]:
tree_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.78,0.75,NaN
F1,0.60,0.59,0.54,NaN
recall,0.49,0.78,NaN,NaN
precision,0.78,0.48,NaN,NaN


In [56]:
results = find_model(features_upsampled, target_upsampled, features_valid, target_valid,
                     model_type='tree', goal='RP', max_depth=11)

tree_results.loc['recall', 'upsamle'] = results[0]
tree_results.loc['precision', 'upsamle'] = results[1]

Полнота (recall): 0.7512315270935961
Точность (precision): 0.41838134430727025


In [57]:
tree_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.78,0.75,NaN
F1,0.60,0.59,0.54,NaN
recall,0.49,0.78,0.75,NaN
precision,0.78,0.48,0.42,NaN


Аналогично логистической регрессии качество решающего дерева upsample не улучшил

**Случайный лес**

In [58]:
results = find_model(features_upsampled, target_upsampled, features_valid, target_valid, model_type='forest', goal='calc')

forest_results.loc['AUC-ROC', 'upsamle'] = results[0]
forest_results.loc['F1', 'upsamle'] = results[1]

[Max ROC-AUC, max_depth, n_estimators]: [0.7891817220982627, 11, 97]
[Max F1, max_depth, n_estimators]: [0.6414602346805737, 18, 97]


In [59]:
forest_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.80,0.79,NaN
F1,0.61,0.65,0.64,NaN
recall,0.49,0.71,NaN,NaN
precision,0.80,0.61,NaN,NaN


In [60]:
results = find_model(features_upsampled, target_upsampled, features_valid, target_valid,
                     model_type='forest', goal='RP', max_depth=17, n_estimators=97)

forest_results.loc['recall', 'upsamle'] = results[0]
forest_results.loc['precision', 'upsamle'] = results[1]

Полнота (recall): 0.6108374384236454
Точность (precision): 0.6595744680851063


In [61]:
forest_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.80,0.79,NaN
F1,0.61,0.65,0.64,NaN
recall,0.49,0.71,0.61,NaN
precision,0.80,0.61,0.66,NaN


Upsample-модель случайного леса чуть хуже автосблансированной модели, но также может в дальнейешем рассматриваться, как один из конечных вариантов

### 2.4.3 Downsample отрицательного класса

In [62]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

**Логистическая регрессия**

In [63]:
results = find_model(features_downsampled, target_downsampled, features_valid, target_valid, model_type='log_reg', goal='calc')

log_reg_results.loc['AUC-ROC', 'downsample'] = results[0]
log_reg_results.loc['F1', 'downsample'] = results[1]

ROC-AUC: 0.6602870369798073
F1 мера: 0.43019296254256534


In [64]:
log_reg_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.59,0.73,0.66,0.66
F1,0.33,0.52,0.43,0.43
recall,0.21,0.73,0.94,NaN
precision,0.69,0.40,0.28,NaN


In [65]:
results = find_model(features_downsampled, target_downsampled, features_valid, target_valid, model_type='log_reg', goal='RP')

log_reg_results.loc['recall', 'downsample'] = results[0]
log_reg_results.loc['precision', 'downsample'] = results[1]

Полнота (recall): 0.9334975369458128
Точность (precision): 0.27949852507374634


In [66]:
log_reg_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.59,0.73,0.66,0.66
F1,0.33,0.52,0.43,0.43
recall,0.21,0.73,0.94,0.93
precision,0.69,0.40,0.28,0.28


Качество модели логистической регрессии после downsaple практически идентично качеству после upsample

**Решающее дерево**

In [67]:
results = find_model(features_downsampled, target_downsampled, features_valid, target_valid, model_type='tree', goal='calc')

tree_results.loc['AUC-ROC', 'downsample'] = results[0]
tree_results.loc['F1', 'downsample'] = results[1]

[Max ROC-AUC, max_depth]: [0.7359494656686714, 2]
[Max F1, max_depth]: [0.5193548387096774, 2]


In [68]:
tree_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.78,0.75,0.74
F1,0.60,0.59,0.54,0.52
recall,0.49,0.78,0.75,NaN
precision,0.78,0.48,0.42,NaN


In [69]:
results = find_model(features_downsampled, target_downsampled, features_valid, target_valid,
                     model_type='tree', goal='RP', max_depth=2)

tree_results.loc['recall', 'downsample'] = results[0]
tree_results.loc['precision', 'downsample'] = results[1]

Полнота (recall): 0.7931034482758621
Точность (precision): 0.38609112709832133


In [70]:
tree_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.78,0.75,0.74
F1,0.60,0.59,0.54,0.52
recall,0.49,0.78,0.75,0.79
precision,0.78,0.48,0.42,0.39


Модель решающего дерева на downsample-данных имеет удовлетворительную точность. При этом f1-мера мало отличается от модели, построенной на upsample-данных. Наиболее характерно для данной модели в данном случае -- малое значение глубины дерева (max_depth == 2)

**Случайный лес**

In [71]:
results = find_model(features_downsampled, target_downsampled, features_valid, target_valid,
                     model_type='forest', goal='calc')

forest_results.loc['AUC-ROC', 'downsample'] = results[0]
forest_results.loc['F1', 'downsample'] = results[1]

[Max ROC-AUC, max_depth, n_estimators]: [0.7209192723946326, 17, 97]
[Max F1, max_depth, n_estimators]: [0.48418334409296315, 17, 97]


In [72]:
forest_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.80,0.79,0.72
F1,0.61,0.65,0.64,0.48
recall,0.49,0.71,0.61,NaN
precision,0.80,0.61,0.66,NaN


In [73]:
results = find_model(features_downsampled, target_downsampled, features_valid, target_valid,
                     model_type='forest', goal='RP', max_depth=16, n_estimators=97)

forest_results.loc['recall', 'downsample'] = results[0]
forest_results.loc['precision', 'downsample'] = results[1]

Полнота (recall): 0.9285714285714286
Точность (precision): 0.3238831615120275


In [74]:
forest_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.80,0.79,0.72
F1,0.61,0.65,0.64,0.48
recall,0.49,0.71,0.61,0.93
precision,0.80,0.61,0.66,0.32


Downsample качество модели случайного леса ухудшил -- точность упала, что уменьшило f1-меру

***Промежуточный итог*** 

In [75]:
log_reg_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.59,0.73,0.66,0.66
F1,0.33,0.52,0.43,0.43
recall,0.21,0.73,0.94,0.93
precision,0.69,0.40,0.28,0.28


In [76]:
tree_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.78,0.75,0.74
F1,0.60,0.59,0.54,0.52
recall,0.49,0.78,0.75,0.79
precision,0.78,0.48,0.42,0.39


In [77]:
forest_results

,unbalanced,auto_balance,upsamle,downsample
AUC-ROC,0.73,0.80,0.79,0.72
F1,0.61,0.65,0.64,0.48
recall,0.49,0.71,0.61,0.93
precision,0.80,0.61,0.66,0.32


In [78]:
log_reg_results['auto_balance']

AUC-ROC     0.73
F1          0.52
recall      0.73
precision   0.40
Name: auto_balance, dtype: object

In [79]:
tree_results['unbalanced']

AUC-ROC     0.73
F1          0.60
recall      0.49
precision   0.78
Name: unbalanced, dtype: object

In [80]:
forest_results['auto_balance']

AUC-ROC     0.80
F1          0.65
recall      0.71
precision   0.61
Name: auto_balance, dtype: object

Исследование по удалению дисбаланса показало, что разные методы балансировки оказывают разное влияние на качество различных моделей.  
По итогам удаления дисбаланса классов целевого признака были выбраны модели ML, обладающие разным качеством работы, а именно:
- модель логистичской регрессии при автобалансировке классов -- f1_мера = 0.52;
- модель решающего дерево на несбалансированных данных -- f1_мера = 0.6;
- модель случайного леса при автобалансировке классов -- f1_мера = 0.64

## 2.5 Тестирование модели

Качество работы выбранных моделей ML будем проверять на тестовом наборе данных. Модель, f1-мера которой будет больше или равна 0.59 (требование Заказчика), будет считаться прошедшей проверку.

In [81]:
test_model(features_train, target_train, features_test, target_test, model_type='log_reg', class_weight='balanced')

F1-мера: 0.492
ROC-AUC: 0.684
recall (полнота): 0.655
precision (точность): 0.393


In [82]:
test_model(features_train, target_train, features_test, target_test, model_type='tree', max_depth=7)

F1-мера: 0.533
ROC-AUC: 0.688
recall (полнота): 0.418
precision (точность): 0.737


In [83]:
test_model(features_train, target_train, features_test, target_test, model_type='forest', class_weight='balanced',
           max_depth=9, n_estimators=97)

F1-мера: 0.618
ROC-AUC: 0.758
recall (полнота): 0.639
precision (точность): 0.598


***Промежуточный итог*** 
  
По итогам проверки работы моделей на тестовом наборе данных наилучшее значение f1-меры принадлежит модели случайного леса, которое равно 0.615. У модели хороший уровень ROC-AUC, который говорит о том, что качество модели заметно лучше случайного классификтора. У модели сбалансированные и заметно высокие значения полноты и точности

# 3 Вывод


В ходе выполнения работы было сделано следующее.  
  
На этапе предобработки данных:
- имена столбцов набора данных были приведены к "under_score" нотации;
- был переименован столбец **Tenure** в **realty_count**, в нём были удалены пропуски;
- бфли удалены столбцы **row_number**, **surname**, **customer_id**;
- было проведено OHE-кодирование для столбцов **geography**, **gender**;
- исходная выборка была разделена на *обучающую*, *валидационную* и *тестовую* выборки в соотношении 3:1:1 (0.6:0.2:0.2) соответственно, выделены из них целевые прзнаки;
- было проведено масштабирование значений стобцов **credit_score**, **age**, **realty_count**, **balance**, **num_of_products**, **estimated_salary** всех выборок.  

Аномальные значения и дубликаты строк в переданном наборе данных отсутствовали.  
  
Затем были построены модели ML логистической регрессии, решающего дерева и случайнго леса на несбалансированных классах целевого признака.  
Полученные модели ML обладали низкой полнотой, т.е. в результатах их работы было много FN значений -- самый негативный вариант работы модели для Заказчика, когда модель прогнозирует, что клиент не уйдёт, а по факут он уходит.  
  
Качество работ моделей необходимо было улучшать. Для этого был убран дисбаланс классов целевого признака.  
  
Дисбаланс убирался при помощи:
- автоматической балансировки классов;
- upsamle положительного класса;  
- downsample отрицательного класса.  
  
Преобразовывалась обучающая выборка с проверкой метрик AUC-ROC, F1 для моделей логистической регрессии, решающего дерева  и случайного леса.  
  
Исследование по удалению дисбаланса показало, что разные методы балансировки оказывают разное влияние на качество различных моделей.  
По итогам удаления дисбаланса классов целевого признака были выбраны модели ML, обладающие разным качеством работы, а именно:
- модель логистичской регрессии при автобалансировке классов -- f1_мера = 0.52;
- модель решающего дерево на несбалансированных данных -- f1_мера = 0.6;
- модель случайного леса при автобалансировке классов -- f1_мера = 0.65.  
  
Тестирование выбранных моделей проводилось на тестовом наборе данных. Модель, f1-мера которой была больше или равна 0.59 (требование Заказчика), считалась прошедшей проверку.  
  
По итогам проверки работы моделей на тестовом наборе данных наилучшее значение f1-меры принадлежит модели случайного леса, которое равно 0.615. У модели хороший уровень ROC-AUC (0.755), который говорит о том, что качество модели заметно лучше случайного классификтора. У модели сбалансированные и заметно высокие значения полноты (0.628) и точности (0.603).  
  
**Таким образом итоговой моделью машинного обучения, имеющей необходимый уровень качества прогнозов является модель случайнго леса на автосбалансированных классах целевого признака, с гиперпараметрами, полученными в ходе выполнения работы**